### Import packages

In [ ]:
import numpy as np
import pandas as pd
#import scipy as sp
from scipy import optimize
from scipy.special import factorial
import statistics as stat
import itertools
!pip install tabulate
from tabulate import tabulate

class par: None
class moms: None

### Define calibration

In [ ]:
### EQUATIONS IN IDENTIFICATION

###### FOR PART 2
def eq_footnote_15(par,moms):
     return moms.TFPgrowth - (par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_T) - (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha))

def eq_15(par,moms):
    par.beta_star = 1/(1+par.r_star)
    return moms.ProfitK - ((par.mu+par.alpha-1)/par.alpha)*(par.r_star + par.delta + par.g_Q/par.beta_star) 

def eq_18(par,moms):
    return moms.XK - ((1+par.g_Q)*(1+par.g_T)-(1-par.delta))

def eq_20(par,moms):
    return moms.ProfitY - (par.mu+par.alpha-1)/(par.mu)

def eq_23(par,moms):
    par.beta_star = 1/(1+par.r_star)
    return moms.PD - par.beta_star*(1+par.g_T)/(1-par.beta_star*(1+par.g_T))
### END OF EQ'S FOR PART 2

###### FOR PART 3 
def find_p(p,par,moms): # Change name from find_p to EQ-number at some point

    update_misc(par)        
    MOM2 = ((1-2*p)+p*np.exp(par.Bh*(1-par.theta)) + p*np.exp(par.B*(1-par.theta)))
    MOM3 = ((1-2*p)+p*np.exp(par.Bh*(-par.theta)) + p*np.exp(par.B*(-par.theta))) 

    return moms.rf - (MOM2/(par.beta_star*MOM3)-1)

def update_misc(par): # Help function for part 3

    par.beta_star = 1/(1+par.r_star)
    par.B = np.log(1-par.b)
    par.Bh = np.log(1+par.b)
    par.g_PC = (1+par.g_T)/(1+par.g_L)-1


###### DEFINE CALIBRATION
def calibrate(ISO,s,e,u,b,theta,sigma):
    #Set country, start years and end years – when calling function, eventually
    country = ISO
    start = s
    end = e
    unique_id = u
    
    # a. shock size
    par.b = b

    # b. risk aversion coefficient
    par.theta = theta

    # c. IES, sigma = 1/IES
    par.sigma = sigma

    #Set data
    df = pd.read_csv("data/data.csv", sep=";", index_col="year")
    df = df[df["ISO"]==country]

    # Calc moments
    moms.ProfitK = stat.mean(df.loc[start:end,"ProfitK"])/100 
    moms.ProfitY = stat.mean(df.loc[start:end,"ProfitY"])/100 
    moms.XK = stat.mean(df.loc[start:end,"XK"])/100
    moms.PD = stat.mean(df.loc[start:end,"PD"])
    moms.TFPgrowth = stat.mean(df.loc[start:end,"TFPgrowth"])/100
    moms.rf = stat.mean(df.loc[start:end,"RF"])/100 #used in step 3
    moms.PopGrowth = stat.mean(df.loc[start:end,"PopGrowth"])/100
    moms.PriceInvt = -stat.mean(df.loc[start:end,"PriceInvt"])/100 # note: negativ value used
    moms.EmpPop = stat.mean(df.loc[start:end,"EmpPop"])/100

    # STEP 1: set parameters g_L, g_Q and N_bar directly
    par.g_L = moms.PopGrowth
    par.g_Q = moms.PriceInvt
    par.N_bar = moms.EmpPop

    # STEP 2

    # Set initial guesses for parameters to be estimated in second part and solve equlation system

    # a. parameters to estimate
    par.names = ['g_Z','g_T','delta','alpha','r_star','mu']

    # b. guess
    par.mu = 1.01
    par.delta = 0.025
    par.alpha = 0.25
    par.g_Z = 0.08 #0.02
    par.r_star = 0.05
    par.g_T = 0.04
    x = set_x(par)

    # c. solve
    solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
    set_parameters(par,solution)

    # STEP 3 – estimate beta and p


    par.p = optimize.fsolve(find_p, 0.1, args=(par,moms), full_output=0)[0]
    update_misc(par)
    MOM =  ((1-2*par.p)+par.p*np.exp(par.Bh*(1-par.theta)) + par.p*np.exp(par.B*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    par.beta = par.beta_star/((1+par.g_PC)**(-par.sigma)*MOM);

    df_estimates = []
    for name in parameters:
        df_estimates.append({'Parameter': name, unique_id:getattr(par,name)})

    df_estimates = pd.DataFrame(df_estimates).set_index('Parameter')
    
    return df_estimates

##### OTHER HELP FUNCTIONS
def set_parameters(par,x):

    for name,value in zip(par.names,x):
        setattr(par,name,value)

def set_x(par):

    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)

    return x

def eq_system(x,par,moms):

    # a. set parameters
    set_parameters(par,x)            

    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_15(par,moms))          
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))
    out.append(eq_23(par,moms))

    return out

### Define decomposition

In [ ]:
def decomp(ISO):
    # Get data
    estimates = pd.DataFrame(index=parameters)
    estimates.index.name = "Name"
    estimates["P1"] = all_estimates[ISO+"_P1"]
    estimates["P2"] = all_estimates[ISO+"_P2"] 

    # Make DataFrame with permutations
    l_permutation = list(itertools.product([0, 1],[0, 1],[0, 1],[0, 1],[0, 1],[0, 1],[0, 1],[0, 1],[0, 1]))
    df_permutation = pd.DataFrame(l_permutation,columns = ['beta','mu','p','delta','alpha','g_L','g_Z','g_Q','N_bar'])
    df_permutation["permsum"] = df_permutation.sum(axis=1)

    # Calculate weights
    for parameter in parameters:
        switch = df_permutation[parameter]
        arr = np.array(df_permutation["permsum"]-switch)
        df_permutation["w_" + parameter] = (factorial(arr)*factorial(8-arr))/factorial(9)

    #Vary all parameters one by one and store values. Save in DataFrame df.
    result = [(ProfitK(par,moms),ProfitY(par,moms),RF(par,moms),PD(par,moms),ik(par,moms),TFPgrowth(par,moms),priceinvt(par,moms),growthpop(par,moms),EmpPop(par,moms))
                for par.beta in [estimates.loc["beta","P1"],estimates.loc["beta","P2"]]
                for par.mu in [estimates.loc["mu","P1"],estimates.loc["mu","P2"]]
                for par.p in [estimates.loc["p","P1"],estimates.loc["p","P2"]]
                for par.delta in [estimates.loc["delta","P1"],estimates.loc["delta","P2"]]
                for par.alpha in [estimates.loc["alpha","P1"],estimates.loc["alpha","P2"]]
                for par.g_L in [estimates.loc["g_L","P1"],estimates.loc["g_L","P2"]]
                for par.g_Z in [estimates.loc["g_Z","P1"],estimates.loc["g_Z","P2"]]
                for par.g_Q in [estimates.loc["g_Q","P1"],estimates.loc["g_Q","P2"]]
                for par.N_bar in [estimates.loc["N_bar","P1"],estimates.loc["N_bar","P2"]]
             ]
    df = pd.DataFrame(result, columns=moments)

    # Join permutations to results
    df = df_permutation.join(df)

    # Take means, etc, to create results
    df_results = pd.DataFrame(columns=parameters)
    df_results["moment"] = moments
    df_results = df_results.set_index("moment",drop=True)

    #Take means of all possible orders, conditional on one parameter. 
    for mom in moments:
        for parm in parameters:
            result = (df.loc[(df[parm] == 1),mom].mul(df["w_"+parm]).sum()-df.loc[(df[parm] == 0),mom].mul(df["w_"+parm]).sum())
            df_results.loc[mom,parm] = result    

    # Format results
    df_results_formatted = df_results.copy()        
    df_results_formatted = df_results_formatted.astype(float) # Convert all to floats (they appear to be strings?)
    df_results_formatted.insert(0,'sum',df_results_formatted.sum(axis=1, skipna=True))    # Sum rows  
    #df_results_formatted['p2'] = [moms.ProfitK,moms.s_K,moms.RF,moms.PD,moms.ik,moms.TFPgrowth,moms.g_Q,moms.g_L,moms.N_bar]
    #df_results_formatted['p1'] = df_results_formatted['p2']-df_results_formatted['change']
    #df_results_formatted = df_results_formatted[["p1","p2","change","beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]] # Change order – sum first


    # Multiply all, except for PD, by 100
    scalar = [1 if i=="PD" else 100 for i in df_results_formatted.index.tolist()]  #Create list, 1 for PD, 100 for all other
    df_results_formatted = df_results_formatted.multiply(scalar,axis=0)

    # Round and set padding zeros
    for var in df_results_formatted.columns.tolist()[:]:
        df_results_formatted[var] = df_results_formatted[var].map('${:,.2f}$'.format)
    return df_results_formatted

# Define functions
def misc(par,moms):
    par.b = -np.log(1-0.15)
    par.bh = -np.log(1+0.15)
    par.sigma = 0.5
    par.theta = 12
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1   
    par.MOM2 = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))
    par.MOM3 = ((1-2*par.p)+par.p*np.exp(-par.bh*(-par.theta)) + par.p*np.exp(-par.b*(-par.theta))) 
    par.MOM = (par.MOM2)**((1-par.sigma)/(1-par.theta))  
    par.beta_star = par.beta * (1+par.g_PC)**(-par.sigma) * par.MOM

#Define equations as functions of parameters only
def growthpop(par,moms):
    return par.g_L

def priceinvt(par,moms):
    return -par.g_Q

def ik(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta)

def EmpPop(par,moms):
    return par.N_bar

def ProfitY(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)

def TFPgrowth(par,moms):
    misc(par,moms)
    moms.ProfitY = (par.mu+par.alpha-1)/(par.mu)
    return par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q)

def ProfitK(par,moms):
    misc(par,moms)
    r_star = 1/par.beta_star - 1
    return ((par.mu + par.alpha -1)/(par.alpha))*(r_star + par.delta + par.g_Q*(1+r_star))

def RF(par,moms):
    misc(par,moms)
    return par.MOM2/(par.MOM3*par.beta_star)-1

def PD(par,moms):
    misc(par,moms)
    return (par.beta_star*(1+par.g_T)/(1-par.beta_star*(1+par.g_T)))

### Settings and assumptions

In [ ]:
moments = ["ProfitK","ProfitY","rf","PD","XK","TFPgrowth","PriceInvt","PopGrowth","EmpPop"]
parameters = ["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]
countries = ["BEL","DNK","FIN","FRA","ITA","JPN","NLD","GBR","USA"]
startP1 = 1984
endP1 = 1999
startP2 = 2000
endP2 = 2015
b = 0.15
theta = 12
sigma = 0.5

# Choose country for specific resulst (tables, decomposition, etc.)
ISO = "GBR"

### Run calibration for all countries

In [ ]:
all_estimates = pd.DataFrame(index=parameters)
for iso in countries:
    for p in ["P1","P2"]:
        if p == "P1": estimates = calibrate(iso,startP1,endP1,iso+"_"+p,b,theta,sigma)
        if p == "P2": estimates = calibrate(iso,startP2,endP2,iso+"_"+p,b,theta,sigma)
        all_estimates[iso+"_"+p] = estimates
        class par: None
        class moms: None
        del estimates
all_estimates

### Print Latex table of estimates

In [ ]:
#ISO = "GBR" # Uncomment to set country here
table = all_estimates.loc[:,ISO+"_P1":ISO+"_P2"].copy()
table['Parameter name'] = ["Discount factor","Mark-up","Disaster probability","Depreciation, pct.","Cobb-Douglas parameter","Population growth, pct.","TFP growth, pct.","Technological change, pct.","Labour Supply"]
table['Symbol'] = ["$\\beta$","$\\mu$","$p$","$\\delta$","$\\alpha$","$g_L$","$g_Z$","$g_Q$","$\\bar{N}$"]
table['Difference'] = table[ISO+"_P2"] - table[ISO+"_P1"]
table = table[["Parameter name","Symbol",ISO+"_P1",ISO+"_P2","Difference"]]
table[ISO+"_P1"] = table[ISO+"_P1"].multiply([1,1,1,100,1,100,100,100,1])
table[ISO+"_P2"] = table[ISO+"_P2"].multiply([1,1,1,100,1,100,100,100,1])
table["Difference"] = table["Difference"].multiply([1,1,1,100,1,100,100,100,1])

for var in table.columns.tolist()[2:]:
    table[var] = table[var].map('${:,.3f}$'.format)

latex = tabulate(table, tablefmt="latex_raw", showindex=False)

print("\\begin{tabular}{lcccr}")
print("\\toprule")
print("& & & \\textit{Estimates} & \\\\ \\cmidrule(lr){3-5}")
print("Parameter name & Symbol & " + str(startP1) + " - " + str(endP1) + " & " + str(startP2) + " - " + str(endP2) + " & Change \\\\")
print("\midrule")
print(latex[30:-21])
print("\\bottomrule")
print("\\end{tabular}")

### Run Decomposition

In [ ]:
#ISO = "GBR" # Uncomment to set country here
decomp(ISO)

### Print Latex table of decomposition

In [ ]:
#ISO = "GBR" # Uncomment to set country here


# UK VALUES HERE – EDIT TO MOMENT dataframe, WHEN CREATED
mom_P1 = [15.631,45.147,5.106,0.255,0.231,26.679,1.363,6.491,44.805]
mom_P2 = [15.630,42.064,1.335,0.652,-0.042,31.775,0.662,6.143,47.249]


formatted_decomp = decomp(ISO).copy()
formatted_decomp.insert(0,"P2",mom_P2)
formatted_decomp.insert(0,"P1",mom_P1)

latex = tabulate(formatted_decomp, showindex=True, tablefmt="latex_raw")

print("\\begin{tabular}{lrrrrrrrrrrrr}")
print("\\toprule")
print(" & \multicolumn{3}{c}{\\textit{Data}} & \multicolumn{9}{c}{\\textit{Decomposition of $\\Delta$}}  \\\\ \\cmidrule(lr){2-4} \\cmidrule(lr){5-13}")
print(" & P1 & P2 & $\\Delta$ & $\\beta$ & $\\mu$ & $p$ & $\\delta$ & $\\alpha$ & $g_L$ & $g_Z$ & $g_Q$ & $\\bar{N}$   \\\\")
print("\\midrule")
print(latex[38:-21])
print("\\bottomrule")
print("\\end{tabular}")